# Single-Cell Report: SCENIC results

In [ ]:
# import dependencies
import os
import numpy as np
import pandas as pd
import scanpy as sc
import loompy as lp
import json
import base64
import zlib
from pyscenic.export import add_scenic_metadata
from pyscenic.cli.utils import load_signatures
import matplotlib as mpl
import matplotlib.pyplot as plt
from scanpy.plotting._tools.scatterplots import plot_scatter
import seaborn as sns
from math import ceil
from IPython.display import Markdown, display

In [ ]:
def printmd(string):
    display(Markdown(string))

#### Plotting settings and functions

In [ ]:
# plot settings
sc.set_figure_params(dpi=150, fontsize=10, dpi_save=600)

In [ ]:
# used for plotting DRs:
def colorMap( x, palette='tab20' ):
    import natsort
    from collections import OrderedDict
    #
    n=len(set(x))
    cpalette = sns.color_palette(palette,n_colors=n )
    cdict = dict( zip( list(set(x)), cpalette ))
    cmap = [ cdict[i] for i in x ]
    cdict = OrderedDict( natsort.natsorted(cdict.items()) )
    return cmap,cdict

def drplot( dr, colorlab, ax, palette='tab20', title=None, **kwargs ):
    cmap,cdict = colorMap( colorlab, palette )
    for lab,col in cdict.items():  
        ix = colorlab.loc[colorlab==lab].index
        ax.scatter( dr['X'][ix], dr['Y'][ix], c=[col]*len(ix), alpha=0.7, label=lab, edgecolors='none')
    if( title is not None ):
        ax.set_title(title, fontsize='x-large');
    #
    ax.set_xticks([])
    ax.set_yticks([])
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)


In [ ]:
# used for heatmap
def palplot(pal, names, colors=None, size=1):
    n = len(pal)
    f, ax = plt.subplots(1, 1, figsize=(n * size, size))
    ax.imshow(np.arange(n).reshape(1, n),
              cmap=mpl.colors.ListedColormap(list(pal)),
              interpolation="nearest", aspect="auto")
    ax.set_xticks(np.arange(n) - .5)
    ax.set_yticks([-.5, .5])
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    colors = n * ['k'] if colors is None else colors
    for idx, (name, color) in enumerate(zip(names, colors)):
        ax.text(0.0+idx, 0.0, name, color=color, horizontalalignment='center', verticalalignment='center')
    return f

In [ ]:
# regulon size barplot
def plotRegulonSize( rlen, title ):
    x=np.arange(rlen.shape[0])

    fig,ax = plt.subplots(figsize=(5,5), dpi=150)
    ax.barh( y=x, width=rlen)

    ax.set_yticks(x)#, rlen.index) #, rotation=0)
    ax.set_yticklabels(rlen.index, minor=False)
    ax.set_xlabel("# of target genes")
    ax.set_ylabel("Transcription factor")
    ax.set_ylim((-1,x.max()+1))
    ax.set_title(title)

    for i, v in enumerate(rlen):
        ax.text(v + 0, i-0.1, str(v), color='blue', fontweight='bold',fontsize=10)

    plt.rc('ytick', labelsize=10)

    plt.tight_layout()
    plt.show()

## Extract relevant data from the integrated loom file

In [ ]:
# scenic output
lf = lp.connect( FILE, mode='r', validate=False )
ca_cell_id = lf.ca.CellID
ca_clusterings = lf.ca.Clusterings
meta = json.loads(lf.attrs["MetaData"])
#exprMat = pd.DataFrame( lf[:,:], index=lf.ra.Gene, columns=lf.ca.CellID).T
if "RegulonsAUC" in lf.ca.keys():
    auc_mtx = pd.DataFrame(lf.ca.RegulonsAUC, index=lf.ca.CellID)
else:
    print("Loom with motif & track regulons detected, merging the regulons AUC matrices...")
    mtf_auc_mtx = pd.DataFrame(lf.ca.MotifRegulonsAUC, index=lf.ca.CellID)
    trk_auc_mtx = pd.DataFrame(lf.ca.TrackRegulonsAUC, index=lf.ca.CellID)
    # merge the AUC matrices:
    auc_mtx = pd.concat([mtf_auc_mtx, trk_auc_mtx], sort=False, axis=1, join='outer')
    # fill NAs (if any) with 0s:
    auc_mtx.fillna(0, inplace=True)

In [ ]:
# create a dictionary of regulons:
regulons = {}
if "Regulons" in lf.ra.keys():
    for i,r in pd.DataFrame(lf.ra.Regulons,index=lf.ra.Gene).iteritems():
        regulons[i] =  list(r[r==1].index.values)
elif "MotifRegulons" in lf.ra.keys() and "TrackRegulons" in lf.ra.keys():
    if "MotifRegulons" in lf.ra.keys():
        for i,r in pd.DataFrame(lf.ra.MotifRegulons,index=lf.ra.Gene).iteritems():
            regulons[i] =  list(r[r==1].index.values)
    if "TrackRegulons" in lf.ra.keys():
        for i,r in pd.DataFrame(lf.ra.TrackRegulons,index=lf.ra.Gene).iteritems():
            regulons[i] =  list(r[r==1].index.values)
else:
    print(f"No regulon information found in the loom: {FILE}...")
    exit(1)

In [ ]:
# capture embeddings:
dr = [
    pd.DataFrame( lf.ca.Embedding, index=lf.ca.CellID )
]
dr_names = [
    meta['embeddings'][0]['name'].replace(" ","_")
]

# add other embeddings
drx = pd.DataFrame( lf.ca.Embeddings_X, index=lf.ca.CellID )
dry = pd.DataFrame( lf.ca.Embeddings_Y, index=lf.ca.CellID )

for i in range( len(drx.columns) ):
    if drx.columns[i] == '-1':
        continue
    dr.append(
        pd.concat(
            [ drx.iloc[:,i], dry.iloc[:,i] ],
            sort=False, axis=1, join='outer' )
    )
    dr_names.append(
        list(filter(lambda e: str(e['id']) == drx.columns[i], meta['embeddings']))[0]['name']
    )

# rename columns:
for i,x in enumerate( dr ):
    x.columns = ['X','Y']

In [ ]:
lf.close()

#### Load this data into a `scanpy.AnnData` object

This can be done directly from the integrated loom file, with a few modifications to allow for SCENIC- and SCope-specific loom attributes:

In [ ]:
adata = sc.read( FILE, validate=False)

adata.obs.drop( ['Embedding','Embeddings_X','Embeddings_Y'], axis=1, inplace=True )
# drop the embeddings and extra attributes from the obs object
if "Regulons" in adata.obs.keys():
    adata.obs.drop( ['RegulonsAUC'], axis=1, inplace=True )
elif "MotifRegulons" in adata.obs.keys() and "TrackRegulons" in adata.obs.keys():
    adata.obs.drop( ['MotifRegulonsAUC', 'TrackRegulonsAUC'], axis=1, inplace=True )
else:
    exit(1)

In [ ]:
# add the embeddings into the adata.obsm object
for i,x in enumerate( dr ):
    adata.obsm[ 'X_'+dr_names[i] ] = x.as_matrix()

In [ ]:
sc.utils.sanitize_anndata( adata )

---
# Regulon summary

In [ ]:
print(
f"There are {len(regulons)} total regulons")

#### Show the top 20 and bottom 20 regulons by size

In [ ]:
rlen = pd.Series( [len(x) for x in regulons.values()], index=regulons.keys() ).sort_values(ascending=True)

plotRegulonSize(rlen.tail(20),"20 largest regulons")
plotRegulonSize(rlen.head(20),"20 smallest regulons")

# Dimensionality reduction plots across all clusterings

### Show all dimensionality reduction plots with all clusterings clustering side-by-side

In [ ]:
for clustering in meta["clusterings"]:
    name = clustering["name"]
    clusters = pd.Series(ca_clusterings[str(clustering["id"])], index=ca_cell_id)
    plt.rcParams.update({'font.size':10})
    fig, (axs) = plt.subplots(ceil(len(dr)/2),2, figsize=(10,10), dpi=100 )
    for i in range(0, len(dr)):
        drplot(
            dr[i], colorlab=clusters, ax=axs[int(i/2)][i%2], palette='tab20', s=2, 
            title=dr_names[i].replace("HVG", "Highly variable genes").replace("_", " ") 
        )
    handles, labels = axs[0][1].get_legend_handles_labels()
    leg = fig.legend(
        handles, labels, loc='upper right', bbox_to_anchor=(1.2, 1), ncol=2, markerscale=2, 
        fontsize='large', frameon=False, title=f"{name}\nclusters"
    )
    fig.suptitle(name, fontsize=14, y=1.02)
    fig.tight_layout()

# Regulon specificity scores (RSS) across all clusterings

In [ ]:
from pyscenic.rss import regulon_specificity_scores
from pyscenic.plotting import plot_rss
import matplotlib.pyplot as plt
from adjustText import adjust_text
import seaborn as sns

### RSS panel plot with all cell types

In [ ]:
for clustering in meta["clusterings"]:
    printmd(f'#### {clustering["name"]}')
    clusters = pd.Series(ca_clusterings[str(clustering["id"])], index=ca_cell_id)
    cats = sorted(list(set(clusters)))
    fig_ncols = 5
    fig_nrows = ceil(len(cats)/fig_ncols)
    fig_asp = 3.75 # final figure aspect ratio

    ## ############
    # Calculate RSS
    rss = regulon_specificity_scores( auc_mtx, clusters )
    
    fig = plt.figure(figsize=(15,15/fig_asp*fig_nrows) )
    for c,num in zip(cats, range(1,len(cats)+1)):
        x=rss.T[c]
        ax = fig.add_subplot(fig_nrows,fig_ncols,num)
        plot_rss(rss, c, top_n=5, max_n=50, ax=ax)
        yr = [ x.sort_values(ascending=False)[:50].min(), x.max() ]
        ax.set_ylim( yr[0] - (yr[1]-yr[0])*0.05 , yr[1] + (yr[1]-yr[0])*0.05 )
        #ax.set_ylim( x.min()-(x.max()-x.min())*0.05 , x.max()+(x.max()-x.min())*0.05 )
        for t in ax.texts:
            t.set_fontsize(12)
        ax.set_ylabel('')
        ax.set_xlabel('')
        adjust_text(ax.texts, autoalign='xy', ha='right', arrowprops=dict(arrowstyle='-',color='lightgrey'), precision=0.001 )

    fig.text(0.5, 0.0, 'Regulon', ha='center', va='center', size='x-large')
    fig.text(0.00, 0.5, 'Regulon specificity score (RSS)', ha='center', va='center', rotation='vertical', size='x-large')
    plt.tight_layout()
    plt.rcParams.update({
        'figure.autolayout': True,
            'figure.titlesize': 'large' ,
            'xtick.labelsize':'large',
            'ytick.labelsize':'large'
            })
    plt.show()

### AUC Heatmap - Top 5 regulons from each cell type

In [ ]:
for clustering in meta["clusterings"]:
    printmd(f'#### {clustering["name"]}')
    clusters = pd.Series(ca_clusterings[str(clustering["id"])], index=ca_cell_id)
    cats = sorted(list(set(clusters)))
    ## ############
    # Calculate RSS
    rss = regulon_specificity_scores( auc_mtx, clusters )
    ###########################################
    # Select top 5 regulons from each cell type
    topreg = []
    for i,c in enumerate(cats):
        topreg.extend(
            list(rss.T[c].sort_values(ascending=False)[:5].index)
        )
    topreg = list(set(topreg))
    ###########################################################################
    # Generate a Z-score for each regulon to enable comparison between regulons
    auc_mtx_Z = pd.DataFrame( index=auc_mtx.index )
    for col in list(auc_mtx.columns):
        auc_mtx_Z[ col ] = ( auc_mtx[col] - auc_mtx[col].mean()) / auc_mtx[col].std(ddof=0)
    #auc_mtx_Z.sort_index(inplace=True)
    ###################
    # Generate heatmap
    # generate a color map:
    cmap,cdict = colorMap( clusters, 'tab20' )
    sns.set()
    sns.set(font_scale=0.8)
    fig = palplot( cdict.values(), cats, size=1.0)
    g = sns.clustermap(
        auc_mtx_Z[topreg].T, annot=False,  square=False, 
        linecolor='gray', xticklabels=False, vmin=-2, vmax=6,
        col_colors=cmap, #colormap,
        cmap="YlGnBu", figsize=(21,16) 
    )
    sns.set(font_scale=1.1)
    g.fig.suptitle(clustering["name"])
    g.cax.set_visible(True)
    g.ax_heatmap.set_ylabel('')    
    g.ax_heatmap.set_xlabel('')